In [2]:
import re

In [3]:
dictionary_lines = {}
def extact_sentences(pattern, file_path):
    # pattern:  is the regular expression you have defined to extract sentences
    # file_path:path to the transcript file 
    
    # key: Jury value: sentence said
    
    for i in pattern:
        with open(file_path, 'r', encoding="unicode_escape") as file:
            line = file.readline()
            while(line):
                # Extract sentences using the regular expression
                match = re.match(i, line)
                if match:
                    if i in dictionary_lines:
                        dictionary_lines[i]+=""+line
                    else:
                        dictionary_lines[i] = line

                line = file.readline()

    # Printing the number of sentences said by each jury (just for checking)
    for jury in dictionary_lines:
        print(jury + ":\t" + str(len(dictionary_lines[jury])))
    print(dictionary_lines.keys())
    return dictionary_lines
path = "/content/12AngryMen.txt"
pattern = [f"NO. {i}" for i in range(2,13)]
res = extact_sentences(pattern,path)

NO. 2:	1103
NO. 3:	9093
NO. 4:	3967
NO. 5:	1821
NO. 6:	1027
NO. 7:	2233
NO. 8:	9544
NO. 9:	1968
NO. 10:	2536
NO. 11:	2176
NO. 12:	930
dict_keys(['NO. 2', 'NO. 3', 'NO. 4', 'NO. 5', 'NO. 6', 'NO. 7', 'NO. 8', 'NO. 9', 'NO. 10', 'NO. 11', 'NO. 12'])


In [4]:
from collections import Counter
def pre_process_inputs(dictionary_lines):
  # This method takes in the dictionary from previous method
  # It will return two dictionaries: one having all the tokens along
  # with their frequencies (list to counter?)
  # The other dictionary will have jury as the key (e.g., NO.2)
  # and the values are another dictionary of {token: frequency} for each jury
  
  lst_all_words = []
  dictionary_juror_words = {}
  for jury in dictionary_lines:
    # Here update all the words list and the dictionary for each jury
    lst_sentences_by_jury = dictionary_lines[jury]
    words_juror = []
    
    words = lst_sentences_by_jury.split()
    words_juror.extend(words)
    lst_all_words.extend(words)
    dictionary_juror_words[jury] = dict(Counter(words_juror)) 
  
  vocabulary = dict(Counter(lst_all_words))
  return vocabulary, dictionary_juror_words
a,b = pre_process_inputs(res)


In [5]:
def unigram_lm(dic_vocabulary, dic_juror_words):
    dic_unigram_lm = {}
    for jury in dic_juror_words:
        dic_words_frequency = dic_juror_words[jury]
        language_model = {key: 0 for key in dic_vocabulary}
        total_words = sum(dic_words_frequency.values())
        for word in dic_vocabulary:
            count_w = dic_words_frequency.get(word, 0)
            language_model[word] = (count_w + 1) / (total_words + len(dic_vocabulary))
        dic_unigram_lm[jury] = language_model
    return dic_unigram_lm
ans = unigram_lm(a,b)

In [7]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
def tsne_generator(dic_unigram_lm):
    tsne = TSNE(n_components=2)
    lst_freq = []
    lst_names = []
    for juror in dic_unigram_lm:
        lst_names.append(juror)
        
        # frequencies is a dictionary of token: frequencies
        # sort this by key and then convert values to the list and append it 
        # to the lst_freq
        frequencies = dic_unigram_lm[juror]
        freq = [frequencies[token] for token in sorted(frequencies.keys())]
        lst_freq.append(freq)
      
    # Convert the list to a NumPy array
    np_array = np.array(lst_freq)
    vectors = tsne.fit_transform(np_array)

    # Create the plot
    fig, ax = plt.subplots()
    ax.scatter(vectors[:, 0], vectors[:, 1])
    for i, txt in enumerate(lst_names):
        ax.annotate(txt, (vectors[i, 0], vectors[i, 1]))
    ax.set_xlabel('t-SNE Dimension 1')
    ax.set_ylabel('t-SNE Dimension 2')
    ax.set_title('t-SNE Plot of Unigram Language Models')
    plt.show()
tsne_generator(ans)
plt.show()

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
